In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
import yaml
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
# from __future__ import division


### Import Toronto data as df,  userJson as df_user

In [2]:
# read csv data reviews 
df_toronto_review = pd.read_csv('../data/toronto_reviews.csv')

# read business csv data 
df_business = pd.read_csv('../data/businesses_final_toronto.csv')

In [3]:
# rename business id
df_business.rename(columns={'id': 'business_id'}, inplace=True)

In [4]:
df_business.head(2)

,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAb...,False,"{'address1': '61 Ossington Avenue', 'address2'...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto...
1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 43.67212500034, 'longitude': -79....",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZ...,False,"{'address1': '2345 Queen Street E', 'address2'...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto...


In [5]:
print("total business numbers:", len(df_business))

total business numbers: 32123


In [6]:
restaurantString = "restaurant|Bar|Bakeries|food|Coffee|Tea|Waffles|Desserts\
|sandwiches|Cupcakes|cafes|Breakfast|Brunch|Lunch|Dinner|chicken|wings\
|singaporean|malaysian|indonesian|russian|taiwanese|american|vietnamese|moroccan\
|cantonese|caribbean|mideastern|asian|italian|korean|himalayan|african|nepalese|laotian|sushi|hawaiian\
|mediterranean|caterers|fish|chips|fishnchips|thai|salad|chinese|seafood|irish|portuguese|french|vegan|raw_food\
|pizza|latin|lebanese|burgers|gluten-free|japanese|pakistani|mexican|sushi|venues|argentine|lounges|barbeque\
|canadian|dimsum|halal|afghan|donuts|juice|smoothies|bagels|ramen|icecream|pubs|falafel\
|noodles|soup|cheese|colombian|turkish|ethiopian|peruvian|hungarian|ukrainian|indian\
|persian|brazilian|vegetarian|austrian|arabian|haitian|cambodian|german|greek|Barbeque"

# We only take restaurant business with the above keywords
cond_category_restaurant = df_business["categories"].str.contains(restaurantString,case=False)

#df_business_Restaurant = df_business[cond_category_restaurant]
df_matched_business = df_business[cond_category_restaurant]

#Now filter out groceries, acupuncture 
filterString = "acupuncture|grocery|bookstores|musicvideo|yoga|servicestations|convenience|homeappliancerepair\
|appliances|Repair|bike|repair|Used|toys|wholesale|supplies|hair|barbers|Hair Salons"

df_business_Restaurant = df_matched_business[~df_matched_business["categories"].str.contains(filterString,case=False)]

print('Matched categories, before filtering size: ', len(df_matched_business))
print('Matched categories, after filtering size: ', len(df_business_Restaurant))


Matched categories, before filtering size:  10131
Matched categories, after filtering size:  9711


In [7]:
#Checking the filtered out businesses at the first step  
cond_category_Not_restaurant = ~df_business["categories"].str.contains(restaurantString,case=False)
business_not_res = df_business[cond_category_Not_restaurant]
# for index, value in business_not_res['categories'].items():
#     print("Index : {}, Value : {}".format(index, value))

In [8]:
#Checking invalid businesses
pd.options.display.max_colwidth = 90  
df_business_Restaurant[df_business_Restaurant["categories"].str.contains('barber',case=False)].categories

Series([], Name: categories, dtype: object)

In [9]:
# for index in range(0,100):
#     print(business_not_res['categories'][index])
#business_not_res['categories']

# for index, value in business_not_res['categories'].items():
#     print("Index : {}, Value : {}".format(index, value))

In [10]:
#Checking business name
# df_business['name'][31600]

In [11]:
# Drop duplicate values in df_business
print(len(df_business_Restaurant))
df_business_clean = df_business_Restaurant.drop_duplicates(subset ="business_id", 
                     keep = False)
print(len(df_business_clean))

9711
9711


## Changing the format of category

In [12]:
# Rename Month as Day and Day as Month(error occurs on original data)
df_toronto_review=df_toronto_review.rename(columns = {'Month':'new_Day'})
df_toronto_review=df_toronto_review.rename(columns = {'Day':'Month'})
df_toronto_review=df_toronto_review.rename(columns = {'new_Day':'Day'})

In [13]:
# df_toronto_review.shape

In [14]:
# df_business_clean.shape

In [15]:
# inner join two full dataset
df = pd.merge(df_toronto_review, df_business_clean, on='business_id', how='inner')

In [16]:
df.shape

(503776, 39)

In [17]:
# merge the date and turn it into string
df['date'] = pd.to_datetime(df[['Year', 'Month','Day']])
df['date'] = df['date'].astype(str)

In [18]:
# rename the columns to match the algorithm 
df.rename(columns={'rating_x': 'stars','rating_y': 'business_stars','review_text': 'text',}, inplace=True)

In [19]:
# get rid of the rows without user_id
df = df[~df["user_id"].isnull()]

In [20]:
pd.set_option('display.max_columns', None)
df.head(2)

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,Unnamed: 0_y,alias,categories,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date
0,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. Lemon mango slush']",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7jZIEA1RZJ6U1s7ZAx3w/348s.jpg'],False,325.0,4.0,122,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,I ordered the lemon mango slush and the lemon taste is very strong. If you love lemon...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}, {'alias': 'coffee', 'title': 'Coffee &...","{'latitude': 43.8258200636421, 'longitude': -79.3063458055258}",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv5Siz8ExZpAC8XSXA/o.jpg,False,"{'address1': 'B25 4300 Steeles Avenue E', 'address2': '', 'address3': None, 'city': 'M...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?adjust_creative=ZbO96qTvvfSmFTkvkUBjkQ&ut...,2016-08-23
1,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. Oreo milk tea', 'Photo of Happy Lemon - ...","['https://s3-media3.fl.yelpcdn.com/bphoto/h110aP3Vldr7KjuhOhryZQ/348s.jpg', 'https://s...",False,307.0,3.0,114,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,"Came here on a Sunday afternoon, it wasn't busy at all. I came here for the Sunday spe...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}, {'alias': 'coffee', 'title': 'Coffee &...","{'latitude': 43.8258200636421, 'longitude': -79.3063458055258}",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv5Siz8ExZpAC8XSXA/o.jpg,False,"{'address1': 'B25 4300 Steeles Avenue E', 'address2': '', 'address3': None, 'city': 'M...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?adjust_creative=ZbO96qTvvfSmFTkvkUBjkQ&ut...,2016-10-02


In [21]:
len(df)

409471

In [22]:
dfff = df.copy()

change category format

In [23]:
df = dfff.copy()

In [24]:
df.reset_index(inplace = True)

In [25]:
ans = []
for cat in tqdm(df.categories.values):
    temp = yaml.safe_load(cat)
#     ans.append([temp[i]["title"] for i in range(len(temp))])
    cat_str = ""
    for i in range(len(temp)):
        if i == 0:
            cat_str += temp[i]["title"]
            if len(temp) >1:
                cat_str += ", "
        elif i == len(temp)-1:
            cat_str += temp[i]["title"]
        else:
            cat_str += temp[i]["title"]
            cat_str += ", "
    ans.append(cat_str)
    

100%|████████████████████████████████████████████████████████████████████████| 409471/409471 [02:31<00:00, 2698.36it/s]


In [26]:
arr_temp = np.asarray(ans)

In [27]:
df_cat = pd.DataFrame(arr_temp)
df_cat.rename(columns={0: "cat"}, inplace = True)

In [28]:
df["cat"] = df_cat

In [29]:
pd.set_option('display.max_rows', 100000)

In [30]:
del df['categories']

In [31]:
df.rename(columns={"cat": "categories"}, inplace = True)

In [32]:
df.head(2)

,index,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,Unnamed: 0_y,alias,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date,categories
0,0,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. Lemon mango slush']",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7jZIEA1RZJ6U1s7ZAx3w/348s.jpg'],False,325.0,4.0,122,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,I ordered the lemon mango slush and the lemon taste is very strong. If you love lemon...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -79.3063458055258}",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv5Siz8ExZpAC8XSXA/o.jpg,False,"{'address1': 'B25 4300 Steeles Avenue E', 'address2': '', 'address3': None, 'city': 'M...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?adjust_creative=ZbO96qTvvfSmFTkvkUBjkQ&ut...,2016-08-23,"Bubble Tea, Coffee & Tea"
1,1,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. Oreo milk tea', 'Photo of Happy Lemon - ...","['https://s3-media3.fl.yelpcdn.com/bphoto/h110aP3Vldr7KjuhOhryZQ/348s.jpg', 'https://s...",False,307.0,3.0,114,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,"Came here on a Sunday afternoon, it wasn't busy at all. I came here for the Sunday spe...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -79.3063458055258}",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv5Siz8ExZpAC8XSXA/o.jpg,False,"{'address1': 'B25 4300 Steeles Avenue E', 'address2': '', 'address3': None, 'city': 'M...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?adjust_creative=ZbO96qTvvfSmFTkvkUBjkQ&ut...,2016-10-02,"Bubble Tea, Coffee & Tea"


In [74]:
df_business_clean.head(6)

,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAbThPHKevj07OijSfA/o.jpg,False,"{'address1': '61 Ossington Avenue', 'address2': '', 'address3': '', 'city': 'Toronto',...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto?adjust_creative=uxtIWjm7I33oX3CDHUtQgQ&...
1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 43.67212500034, 'longitude': -79.28946080056}",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZr-V8-VJHNLQJyvgQ/o.jpg,False,"{'address1': '2345 Queen Street E', 'address2': '', 'address3': '', 'city': 'Toronto',...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto-3?adjust_creative=9LfdvYfx0rVPnDmG5bc0i...
2,2,subway-toronto-51,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}, {'alias': 'hotdogs', 'title': 'Fast F...","{'latitude': 43.66449, 'longitude': -79.34116}",+1 416-465-8100,2934.260861,f8YPC7K4UUTkKatE5ga8Ig,https://s3-media1.fl.yelpcdn.com/bphoto/JpxU9FOgghpQQptpdsDDtg/o.jpg,False,"{'address1': '331 Carlaw Avenue', 'address2': 'Unit 103', 'address3': '', 'city': 'Tor...",Subway,1.416466e+10,$$,1.5,6,[],https://www.yelp.com/biz/subway-toronto-51?adjust_creative=NeTgDT5IUqTFt2XAEV__OA&utm_...
5,5,jodies-restaurant-toronto,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {'alias': 'diners', 'title': 'Diners'}, {...","{'latitude': 43.6053352355957, 'longitude': -79.5289154052734}",+1 416-255-5552,4262.777626,K5TteBnAfgj18HRnV-Yy4w,https://s3-media3.fl.yelpcdn.com/bphoto/GtkSIOchBTNry8WbIiZ7Gg/o.jpg,False,"{'address1': '331 Horner Avenue', 'address2': '', 'address3': '', 'city': 'Toronto', '...",Jodie's Restaurant,1.416256e+10,$$,4.5,6,[],https://www.yelp.com/biz/jodies-restaurant-toronto?adjust_creative=uxtIWjm7I33oX3CDHUt...
11,11,the-great-canadian-bagel-ltd-vaughan,"[{'alias': 'bagels', 'title': 'Bagels'}]","{'latitude': 43.7928079, 'longitude': -79.5312639}",+1 905-760-9955,7595.423171,ARwwcafc0x5s9wC-Te9yXg,NaN,False,"{'address1': '3300 Hwy 7', 'address2': 'Suite 101', 'address3': '', 'city': 'Vaughan',...","The Great Canadian Bagel, Ltd",1.905761e+10,$$,4.0,1,[],https://www.yelp.com/biz/the-great-canadian-bagel-ltd-vaughan?adjust_creative=9LfdvYfx...
18,18,tim-hortons-toronto-240,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'bakeries', 'title': 'Bakerie...","{'latitude': 43.61003, 'longitude': -79.55318}",+1 416-521-7336,2661.026341,ym-vQYhSONsViLfxbXJ0_g,https://s3-media3.fl.yelpcdn.com/bphoto/0f6QI_f_0rLQwJXPLPVhXA/o.jpg,False,"{'address1': '660 Evans Avenue', 'address2': '', 'address3': None, 'city': 'Toronto', ...",Tim Horton's,1.416522e+10,NaN,3.5,2,[],https://www.yelp.com/biz/tim-hortons-toronto-240?adjust_creative=ZbO96qTvvfSmFTkvkUBjk...


In [38]:
yaml.safe_load(df_location[df_location["business_num_id"] == prediction_matrix[user_id][j]].iloc[0].coordinates)


str

In [41]:

yaml.safe_load(df_business_clean.coordinates[0])

{'latitude': 43.6456, 'longitude': -79.41903}

In [86]:
df_business_clean = df_business_clean.reset_index()

In [88]:
latitudeList = []
longitudeList = []

for business in range(len(df_business_clean)):
    latitude= yaml.safe_load(df_business_clean.coordinates[business])['latitude']
    longitude = yaml.safe_load(df_business_clean.coordinates[business])['longitude']
    latitudeList.append(latitude)
    longitudeList.append(longitude)

In [90]:
df_latitude = pd.DataFrame(np.asarray(latitudeList))
df_latitude.rename(columns={0: "latitude"}, inplace = True)

In [91]:
df_longitude = pd.DataFrame(np.asarray(longitudeList))
df_longitude.rename(columns={0: "longitude"}, inplace = True)

In [93]:
df_business_clean["latitude"] = df_latitude
df_business_clean["longitude"] = df_longitude

In [94]:
df_business_clean.head(2)

,index,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,latitude,longitude
0,0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAbThPHKevj07OijSfA/o.jpg,False,"{'address1': '61 Ossington Avenue', 'address2': '', 'address3': '', 'city': 'Toronto',...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto?adjust_creative=uxtIWjm7I33oX3CDHUtQgQ&...,43.6456,-79.41903
1,1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 43.67212500034, 'longitude': -79.28946080056}",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZr-V8-VJHNLQJyvgQ/o.jpg,False,"{'address1': '2345 Queen Street E', 'address2': '', 'address3': '', 'city': 'Toronto',...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto-3?adjust_creative=9LfdvYfx0rVPnDmG5bc0i...,43.67212500034,-79.28946080056


### Data Processing

In [34]:
Export = df.to_json(r'..\\data\\Export_TorontoData.json', orient='records')

In [95]:
Export = df_business_clean.to_json(r'..\\data\\Export_ClearnTorontoBusiness.json', orient='records')